### Finding power series expansions along exceptional components
------------------------------------------------------------------------------------------
First we define the surface, for now only in affine form. This is the surface Wittenberg works with in his article

In [ ]:
A3<x,y,t>:=AffineSpace(Rationals(),3);
p:=3*(t-1)^3*(t+3);
q:=Evaluate(p,[x,y,-t]);
F:=y^2-x*(x-p)*(x-q);
E:=Surface(A3,F);

We also must compute the equations for $E$ on the affine patch about infinity.

In [ ]:
p_inf:=3*(1-t)^3*(1+3*t);
q_inf:=3*(-1-t)^3*(-1+3*t);
F_inf:=y^2-x*(x-p_inf)*(x-q_inf);
E_inf:=Surface(A3, F_inf);

We check the discriminant of the elliptic curve. We need to do some hacky algebra to extract the polynomial we're interested in.

In [ ]:
Factorization(Discriminant(F-y^2,x));
Factorization(Discriminant(F_inf-y^2,x));

Singular subscheme, and some of the points in the support

In [ ]:
Support(SingularSubscheme(E));
Support(SingularSubscheme(E_inf));

In [ ]:
Etilde:=DesingulariseSurfaceByBlowUp(E);
Length(Etilde[2]);
Etilde_inf:=DesingulariseSurfaceByBlowUp(E_inf);

### Residue formula

Given a complete local DVR $R$ with uniformizer $\pi$ and residue field $k$, we compute the residue of a quaternion algebra $(f,g)$ as the class of $(-1)^{v(f)v(g)}\widetilde{f}^{v(g)}\widetilde{g}^{v(f)}$ in $k^*/k^{*2}$ where for any $h\in R$ we define $\widetilde{h}=h\pi^{-v(h)}$.

In [ ]:
function ComputeResidue(f, g)
    vf:=Valuation(f);
    vg:=Valuation(g);
    
    f_0:=Coefficients(f)[1];
    g_0:=Coefficients(g)[1];
    
    return f_0^(vg)*g_0^(vf)*(-1)^(vf*vg);
end function;

function IsSquareQU(f)
    R := PolynomialRing(Integers());
    return IsSquare(R ! Numerator(f)) and IsSquare(R ! Denominator(f));
end function;

function ComputeAllInterestingResidues(X, T, IsInf)
    if IsInf then
        PT := 3*(1-T)^3*(1+3*T);
        QT := 3*(-1-T)^3*(-1+3*T);
    else
        PT := 3*(T-1)^3*(T+3);
        QT := 3*(-T-1)^3*(-T+3);
    end if;
    Glist := [X-PT, X-QT];
    Flist := [T, T+1, T-1, T+3, T-3];
    if IsInf then
        Flist := [T, 1+T, 1-T, 1+3*T, 1-3*T];
    end if;
    return [ [ ComputeResidue(f,g) : f in Flist ] : g in Glist];
end function;

## Computing list of residues for all bad fibers

### Computing residue at ((-9, 0), 0) 

#### Component 1

Here we are computing the residue along the original singular fiber $\mathcal{E}_0$.

In [ ]:
A2<x2,y2>:=AffinePlane(Rationals());
L:=Curve(A2, Evaluate(DefiningPolynomial(E), [x2,y2,0]));
L;

We define a rational function field that we use to parametrize the exceptional component L with

In [ ]:
kU<U>:=FunctionField(Rationals(), 1);

We write down a parametrization of L and check that it satisfies that parametrization. In particular, we see that "V" on Y vanishes on L (we'll see we can use it as a uniformizer at the generic point of L)

In [ ]:
Xu:=U^2;
Yu:=U^3+9*U;
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[Xu,Yu]) eq 0};

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Tu:=Vu;

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
ResidueArray:=AssociativeArray(Integers());
ResidueArray[0]:=AssociativeArray(Integers());
ResidueArray[0][1]:=Residues;

#### Component 2

Here we unpack the equations for the first exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((-9, 0),0)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[1],1);
A3<T,S,W>:=Ambient(Y);

We define a rational function field that we use to parametrize the exceptional component L with

In [ ]:
kU<U>:=FunctionField(Rationals(),1);

We write down a parametrization of L and check that it satisfies that parametrization. In particular, we see that "V" on Y vanishes on L (we'll see we can use it as a uniformizer at the generic point of L)

In [ ]:
Su:=3/2*(U^2-1)/U;
Wu:=1/48*(U^2+1)/U;
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[0,Su,Wu]) eq 0};

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Tu:=Vu;

In [ ]:
R<Wpol>:=PolynomialRing(kUV);

Below we compute a higher order approximation of $W$ in the power series ring $k(U)[[V]]$.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Tu, Su, Wpol]);
Fprime:=Derivative(F);
for i in [1..4] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series in $k(U)[[V]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Tu,Su,Wu])+O(Tu^(10));

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,T$ in terms of $T,S,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Tu, Vu, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Tu, Vu, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
ResidueArray[0][2]:=Residues;

### Computing residue at ((0, 0), -3) 

#### Component 1

Here we are computing the residue along the original singular fiber $\mathcal{E}_{-3}$.

In [ ]:
A2<x2,y2>:=AffinePlane(Rationals());
L:=Curve(A2, Evaluate(DefiningPolynomial(E), [x2,y2,-3]));
L;

We define a rational function field that we use to parametrize the exceptional component L with

In [ ]:
kU<U>:=FunctionField(Rationals(), 1);

We write down a parametrization of L and check that it satisfies that parametrization.

In [ ]:
Xu:=U^2+144;
Yu:=U^3+144*U;
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[Xu,Yu]) eq 0};

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Tu:=Vu-3;

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
ResidueArray[-3]:=AssociativeArray(Integers());
ResidueArray[-3][1]:=Residues;

#### Component 2

Here we unpack the equations for the first exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), 3)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[2],1);
A3<X,S,W>:=Ambient(Y);
L;

We define a rational function field that we use to parametrize the exceptional component L with

In [ ]:
kU<U>:=FunctionField(Rationals(),1);

Below we find a parametrization of $L$.

In [ ]:
Lc,phiL:=Conic(ProjectiveClosure(L));
piL:=Parametrization(Lc);
Q<x, s, w>:=FunctionField(L);
kU<U>:=FunctionField(Domain(piL));
Su:=Pullback(piL, Pullback(Inverse(phiL), s));
Wu:=Pullback(piL, Pullback(Inverse(phiL), w));
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[0,Su,Wu]) eq 0};

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Xu:=Vu;

Below we compute a higher order approximation of $W$ in the power series ring $k(U)[[V]]$.

In [ ]:
R<Wpol>:=PolynomialRing(kUV);
F:=Evaluate(DefiningPolynomial(Y), [Xu, Su, Wpol]);
Fprime:=Derivative(F);
for i in [1..4] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series in $k(U)[[V]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Xu,Su,Wu])+O(Vu^8);

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Xu, Su, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Xu, Su, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
ResidueArray[-3][2]:=Residues;

### Computing residues at ((0, 0), -1) 

#### Component 1

Here we compute the residue at the original component of $\mathcal{E}_{-1}$

In [ ]:
A2<x2,y2>:=AffinePlane(Rationals());
L:=Curve(A2, Evaluate(DefiningPolynomial(E), [x2,y2,-1]));
L;

We define a rational function field that we use to parametrize the exceptional component L with

In [ ]:
kU<U>:=FunctionField(Rationals(), 1);

We write down a parametrization of L and check that it satisfies that parametrization. In particular, we see that "V" on Y vanishes on L (we'll see we can use it as a uniformizer at the generic point of L)

In [ ]:
Xu:=U^2-48;
Yu:=U^3-48*U;
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[Xu,Yu]) eq 0};

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Tu:=Vu-1;

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,T$ in terms of $V,W,T$ (apologies for the mess-up in naming here: the two $T$s are not the same), and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
ResidueArray[-1]:=AssociativeArray(Integers());
ResidueArray[-1][1]:=Residues;

#### Component 2

Here we unpack the equations for the first exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), -1)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[3],1);

Looking at the equation of $L$ we note that it has no rational points and is geometrically reducible. This means that the residue field $k(L)$ will be of the form $k(u)$ with $k$ some finite extension of $\mathbb{Q}$; in our case $k=\mathbb{Q}(\sqrt{48})=\mathbb{Q}(\sqrt{3})$.

In [ ]:
A3<X,V,W>:=Ambient(Y);
L;

Note that $L:V^2-48=0, X=0$ has no rational points and is geometrically reducible. In particular, one can work out that the residue field is $\kappa(L)=k(W)$ where $k=\mathbb{Q}(a)$ and $a$ is a square root of $3$. Below we define the extension $k/\mathbb{Q}$ as well as a rational function field that we will use to parametrize the exceptional component L with.

In [ ]:
k<a>:=QuadraticField(48);
kU<U>:=FunctionField(k,1);
Wu:=U;

In [ ]:
kUX<Xu>:=PowerSeriesRing(kU);

In [ ]:
R<Vpol>:=PolynomialRing(kUX);

By definition we have an expression for $X$ and $W$ in $k(U)(Xu)$ as $W=U$ and $X=Xu$. It remains to find an expansion for $V$. From our observations above we note that when taking residues $\overline{V}=4a=4\sqrt{3}\in k(W)$, as we mod out by $V^2-48$, and so we have a first order approximation. Applying Newton iteration gives us enough terms to compute residues.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Xu, Vpol, Wu]);
Fprime:=Derivative(F);
Vu:=4*a;
for i in [1..4] do
    Temp:=Vu-Evaluate(F, Vu)/Evaluate(Fprime,Vu);
    Vu:=Temp;
end for;

We verify that the power series $Xu,Vu,Wu$ in $k(U)[[Xu]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Xu,Vu,Wu])+O(Xu^10);

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,T$ in terms of $X,V,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Xu, Vu, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Xu, Vu, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
ResidueArray[-1][2]:=Residues;

#### Component 3

Here we unpack the equations for the second exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), -1)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[3],3);

Looking at the equation of $L$ we note that it has no rational points and is geometrically reducible. This means that the residue field $k(L)$ will be of the form $k(u)$ with $k$ some finite extension of $\mathbb{Q}$; in our case $k=\mathbb{Q}(\sqrt{48})=\mathbb{Q}(\sqrt{3})$.

In [ ]:
A3<X,S,W>:=Ambient(Y);
L;

We define the extension $k/\mathbb{Q}$ as well as a rational function field that we will use to parametrize the exceptional component L with.

In [ ]:
kU<U>:=FunctionField(Rationals(), 1);
Su:=12*U^2/(-U^2+1/48);
Wu:=12*U/(-U^2+1/48);
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[-1,Su,Wu]) eq 0};

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Xu:=Vu-1;

In [ ]:
R<Wpol>:=PolynomialRing(kUV);

By definition we have an expression for $X$ and $W$ in $k(U)(Xu)$ as $W=U$ and $X=Xu$. It remains to find an expansion for $V$. We note that when taking residues we have $\overline{V}=4a=4\sqrt{3}$, as we mod out by $V^2-48$, and so we have a first order approximation. Applying Newton iteration gives us enough terms to compute residues.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Xu, Su, Wpol]);
Fprime:=Derivative(F);
for i in [1..2] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series $Xu,Vu,Wu$ in $k(U)[[Xu]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Xu,Su,Wu])+O(Vu^6);

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,T$ in terms of $X,V,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Xu, Su, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Xu, Su, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
ResidueArray[-1][3]:=Residues;

#### Component 4

Here we unpack the equations for the third exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), -1)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[3],3);

In [ ]:
A3<T,V,W>:=Ambient(Y);
L;

We write down a parametrization of our conic $L$. For the parametrization below we took the projective closure and projected away from the rational point $[0,0,1]$.

In [ ]:
kU<U>:=FunctionField(Rationals(), 1);
Vu:=(-6*U^2-1/8)/(-U^2+1/48)-6;
Wu:=-12*U/(-U^2+1/48);
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[-1,Vu,Wu]) eq 0};

In [ ]:
kUT<Tu>:=PowerSeriesRing(kU);

In [ ]:
R<Wpol>:=PolynomialRing(kUT);

Now we compute an expansion for $W$ in $\mathbb{Q}(U)[[T]]$.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Tu-1, Vu, Wpol]);
Fprime:=Derivative(F);
for i in [1..4] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series $Tu-1,Vu,Wu$ in $k(U)[[Tu]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Tu-1,Vu,Wu])+O(Tu^8);

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,T$ in terms of $X,V,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Tu, Vu, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Tu, Vu, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
ResidueArray[-1][4]:=Residues;

### Computing residues at ((0, 0), 1) 

#### Component 1

Here we compute the residues over the original component $\mathcal{E}_1$.

In [ ]:
A2<x2,y2>:=AffinePlane(Rationals());
L:=Curve(A2, Evaluate(DefiningPolynomial(E), [x2,y2,1]));
L;

We define a rational function field that we use to parametrize the exceptional component L with

In [ ]:
Q<U>:=FunctionField(Rationals(), 1);
R<y>:=PolynomialRing(Q);
kU<a>:=quo<R | y^2-U-48>;

We write down a parametrization of L and check that it satisfies that parametrization. In particular, we see that "V" on Y vanishes on L (we'll see we can use it as a uniformizer at the generic point of L)

In [ ]:
Xu:=U;
Yu:=U*a;
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[Xu,Yu]) eq 0};

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Tu:=Vu+1;

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
ResidueArray[1]:=AssociativeArray(Integers());
ResidueArray[1][1]:=Residues;

#### Component 2

Here we unpack the equations for the first exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), 1)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[4],1);

Looking at the equation of $L$ we note that it has no rational points and is geometrically reducible. This means that the residue field $k(L)$ will be of the form $k(u)$ with $k$ some finite extension of $\mathbb{Q}$; in our case $k=\mathbb{Q}(\sqrt{48})=\mathbb{Q}(\sqrt{3})$.

In [ ]:
A3<X,V,W>:=Ambient(Y);
L;

Note that $L:V^2-48=0, X=0$ has no rational points and is geometrically reducible. In particular, one can work out that the residue field is $\kappa(L)=k(W)$ where $k=\mathbb{Q}(a)$ and $a$ is a square root of $3$. Below we define the extension $k/\mathbb{Q}$ as well as a rational function field that we will use to parametrize the exceptional component L with.

In [ ]:
k<a>:=QuadraticField(48);
kU<U>:=FunctionField(k,1);
Wu:=U;

In [ ]:
kUX<Xu>:=PowerSeriesRing(kU);

In [ ]:
R<Vpol>:=PolynomialRing(kUX);

By definition we have an expression for $X$ and $W$ in $k(U)(Xu)$ as $W=U$ and $X=Xu$. It remains to find an expansion for $V$. From our observations above we note that when taking residues $\overline{V}=4a=4\sqrt{3}\in k(W)$, as we mod out by $V^2-48$, and so we have a first order approximation. Applying Newton iteration gives us enough terms to compute residues.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Xu, Vpol, Wu]);
Fprime:=Derivative(F);
Vu:=4*a;
for i in [1..4] do
    Temp:=Vu-Evaluate(F, Vu)/Evaluate(Fprime,Vu);
    Vu:=Temp;
end for;

We verify that the power series $Xu,Vu,Wu$ in $k(U)[[Xu]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Xu,Vu,Wu])+O(Xu^10);

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,T$ in terms of $X,V,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Xu, Vu, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Xu, Vu, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
ResidueArray[1][2]:=Residues;
Residues;

#### Component 3

Here we unpack the equations for the second exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), 1)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[4],2);

Looking at the equation of $L$ we note that it has no rational points and is geometrically reducible. This means that the residue field $k(L)$ will be of the form $k(u)$ with $k$ some finite extension of $\mathbb{Q}$; in our case $k=\mathbb{Q}(\sqrt{48})=\mathbb{Q}(\sqrt{3})$.

In [ ]:
A3<X,V,W>:=Ambient(Y);
L;

We define the extension $k/\mathbb{Q}$ as well as a rational function field that we will use to parametrize the exceptional component L with.

In [ ]:
k<a>:=QuadraticField(48);
kU<U>:=FunctionField(k,1);
Vu:=U;

In [ ]:
kUX<Xu>:=PowerSeriesRing(kU);

In [ ]:
R<Wpol>:=PolynomialRing(kUX);

By definition we have an expression for $X$ and $W$ in $k(U)(Xu)$ as $W=U$ and $X=Xu$. It remains to find an expansion for $V$. We note that when taking residues we have $\overline{V}=4a=4\sqrt{3}$, as we mod out by $V^2-48$, and so we have a first order approximation. Applying Newton iteration gives us enough terms to compute residues.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Xu, Vu, Wpol]);
Fprime:=Derivative(F);
Wu:=4*a;
for i in [1..4] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series $Xu,Vu,Wu$ in $k(U)[[Xu]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Xu,Vu,Wu])+O(Xu^10);

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,T$ in terms of $X,V,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Xu, Vu, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Xu, Vu, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
ResidueArray[1][3]:=Residues;

#### Component 4

Here we unpack the equations for the third exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), 1)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[4],3);

In [ ]:
A3<T,V,W>:=Ambient(Y);
L;

We write down a parametrization of our conic $L$. For the parametrization below we took the projective closure and projected away from the rational point $[0,0,1]$.

In [ ]:
kU<U>:=FunctionField(Rationals(), 1);
Vu:=(-6*U^2-1/8)/(-U^2+1/48)+6;
Wu:=-12*U/(-U^2+1/48);
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[1,Vu,Wu]) eq 0};

In [ ]:
kUT<Tu>:=PowerSeriesRing(kU);

In [ ]:
R<Wpol>:=PolynomialRing(kUT);

Now we compute an expansion for $W$ in $\mathbb{Q}(U)[[T]]$.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Tu+1, Vu, Wpol]);
Fprime:=Derivative(F);
for i in [1..4] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series $Tu-1,Vu,Wu$ in $k(U)[[Tu]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Tu+1,Vu,Wu])+O(Tu^8);

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,T$ in terms of $X,V,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Tu, Vu, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Tu, Vu, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
ResidueArray[1][4]:=Residues;

### Computing residue at ((0, 0), 3) 

#### Component 1

Here we compute the residues over the original component $\mathcal{E}_3$.

In [ ]:
A2<x2,y2>:=AffinePlane(Rationals());
L:=Curve(A2, Evaluate(DefiningPolynomial(E), [x2,y2,3]));
L;

We define a rational function field that we use to parametrize the exceptional component L with

In [ ]:
Q<U>:=FunctionField(Rationals(), 1);
R<y>:=PolynomialRing(Q);
kU<a>:=quo<R | y^2-U+144>;

We write down a parametrization of L and check that it satisfies that parametrization. In particular, we see that "V" on Y vanishes on L (we'll see we can use it as a uniformizer at the generic point of L)

In [ ]:
Xu:=U;
Yu:=U*a;
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[Xu,Yu]) eq 0};

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Tu:=Vu+3;

In [ ]:
Residues:=ComputeAllInterestingResidues(Xu, Tu, false);
ResidueArray[3]:=AssociativeArray(Integers());
ResidueArray[3][1]:=Residues;

#### Component 2

Here we unpack the equations for the first exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), 1)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[5],1);
A3<X,V,W>:=Ambient(Y);

We define a rational function field that we use to parametrize the exceptional component L with

In [ ]:
kU<U>:=FunctionField(Rationals(),1);
L;

Below we find a parametrization of $L$.

In [ ]:
Lc,phiL:=Conic(ProjectiveClosure(L));
piL:=Parametrization(Lc);
Q<x, v, w>:=FunctionField(L);
kU<U>:=FunctionField(Domain(piL));
Vu:=Pullback(piL, Pullback(Inverse(phiL), v));
Wu:=Pullback(piL, Pullback(Inverse(phiL), w));
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[0,Vu,Wu]) eq 0};

In [ ]:
kUX<Xu>:=PowerSeriesRing(kU);

In [ ]:
R<Wpol>:=PolynomialRing(kUX);

Here we compute a higher order approximation for $W$ in the power series ring.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Xu, Vu, Wpol]);
Fprime:=Derivative(F);
for i in [1..4] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series in $k(U)[[V]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Xu,Vu,Wu])+O(Xu^8);

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Xu, Vu, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Xu, Vu, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
ResidueArray[3][2]:=Residues;

### Computing residues at ((3, 0), $\infty$) 

#### Component 1

Here we compute the residues over the original component $\mathcal{E}_{\infty}$.

In [ ]:
A2<x2,y2>:=AffinePlane(Rationals());
L:=Curve(A2, Evaluate(DefiningPolynomial(E_inf), [x2,y2,0]));
L;

We define a rational function field that we use to parametrize the exceptional component L with

In [ ]:
Q<U>:=FunctionField(Rationals(), 1);
R<y>:=PolynomialRing(Q);
kU<a>:=quo<R | y^2-U^3+6*U^2-9*U>;

We write down a parametrization of L and check that it satisfies that parametrization. In particular, we see that "V" on Y vanishes on L (we'll see we can use it as a uniformizer at the generic point of L)

In [ ]:
Xu:=U;
Yu:=a;
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[Xu,Yu]) eq 0};

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Su:=Vu;

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Su, true);
ResidueArray[10]:=AssociativeArray(Integers());
ResidueArray[10][1]:=Residues;

#### Component 2

Here we unpack the equations for the first exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), 1)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E_inf,Etilde_inf[1],1);

Looking at the equation of $L$ we note that it has no rational points and is geometrically reducible. This means that the residue field $k(L)$ will be of the form $k(u)$ with $k$ some finite extension of $\mathbb{Q}$; in our case $k=\mathbb{Q}(\sqrt{3})$.

In [ ]:
A3<S,V,W>:=Ambient(Y);
L;

The variable $S$ corresponds to $1/t$ in our original coordinates.

Note that $L:V^2-3=0, S=0$ has no rational points and is geometrically reducible. In particular, one can work out that the residue field is $\kappa(L)=k(W)$ where $k=\mathbb{Q}(a)$ and $a$ is a square root of $3$. Below we define the extension $k/\mathbb{Q}$ as well as a rational function field that we will use to parametrize the exceptional component L with.

In [ ]:
k<a>:=QuadraticField(3);
kU<U>:=FunctionField(k,1);
Wu:=U;

In [ ]:
kUS<Su>:=PowerSeriesRing(kU);

In [ ]:
R<Vpol>:=PolynomialRing(kUS);

By definition we have an expression for $S$ and $W$ in $k(U)(Su)$ as $W=U$ and $S=Xu$. It remains to find an expansion for $V$. From our observations above we note that when taking residues $\overline{V}=a=\sqrt{3}\in k(W)$, as we mod out by $V^2-3$, and so we have a first order approximation. Applying Newton iteration gives us enough terms to compute residues.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Su, Vpol, Wu]);
Fprime:=Derivative(F);
Vu:=a;
for i in [1..4] do
    Temp:=Vu-Evaluate(F, Vu)/Evaluate(Fprime,Vu);
    Vu:=Temp;
end for;

We verify that the power series $Su,Vu,Wu$ in $k(U)[[Su]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Su,Vu,Wu])+O(Su^10);

In order to relate all this back to the original coordinates on $E$, we use the description of pi, that expresses $X,Y,T$ in terms of $X,V,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Su, Vu, Wu]);
Su:=Evaluate(DefiningPolynomials(pi)[3], [Su, Vu, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Su, true);
ResidueArray[10][2]:=Residues;

#### Patch 3

Here we unpack the equations for the second exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), 1)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E_inf,Etilde_inf[1],2);

Looking at the equation of $L$ we note that it has no rational points and is geometrically reducible. This means that the residue field $k(L)$ will be of the form $k(u)$ with $k$ some finite extension of $\mathbb{Q}$; in our case $k=\mathbb{Q}(\sqrt{3})$.

In [ ]:
A3<S,V,W>:=Ambient(Y);
L;
Factorization(972);

Here we see that $L:W^2/972-(V+1/18)^2=0$ and therefore $L$ reducible to the product of two lines over $\mathbb{Q}(\sqrt{972})=\mathbb{Q}(\sqrt{3})$. Thus the residue field $\kappa(L)$ is $\mathbb{Q}(\sqrt{3})(U)$ with $U=V$.

In [ ]:
k<a>:=QuadraticField(3);
kU<U>:=FunctionField(k,1);
Vu:=U;
Wu:=(18*Vu+1)*a;

In [ ]:
kUS<Su>:=PowerSeriesRing(kU);

In [ ]:
R<Wpol>:=PolynomialRing(kUS);

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Su, Vu, Wpol]);
Fprime:=Derivative(F);
for i in [1..4] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series $Su,Vu,Wu$ in $k(U)[[Su]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Su,Vu,Wu])+O(Su^10);

In order to relate all this back to the original coordinates on $E$, we use the description of pi, that expresses $X,Y,T$ in terms of $X,V,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Su, Vu, Wu]);
Su:=Evaluate(DefiningPolynomials(pi)[3], [Su, Vu, Wu]);

In [ ]:
Residues:=ComputeAllInterestingResidues(Xu, Su, true);
ResidueArray[10][3]:=Residues;

#### Component 4

Here we unpack the equations for the third exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), 1)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E_inf,Etilde_inf[1],3);

In [ ]:
A3<S,V,W>:=Ambient(Y);
L;

We write down a parametrization of our conic $L$. For the parametrization below we took the projective closure and projected away from the rational point $[24,0,1]$.

In [ ]:
kU<U>:=FunctionField(Rationals(), 1);
Vu:=(24*U^2+8)/(-U^2+1/3);
Wu:=48*U/(-U^2+1/3);
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[0,Vu,Wu]) eq 0};

In [ ]:
kUS<Su>:=PowerSeriesRing(kU);

In [ ]:
R<Wpol>:=PolynomialRing(kUS);

Now we compute an expansion for $W$ in $\mathbb{Q}(U)[[S]]$.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Su, Vu, Wpol]);
Fprime:=Derivative(F);
for i in [1..4] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series $Su,Vu,Wu$ in $k(U)[[Tu]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Su,Vu,Wu])+O(Su^8);

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,S$ in terms of $X,V,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Su, Vu, Wu]);
Su:=Evaluate(DefiningPolynomials(pi)[3], [Su, Vu, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Su, true);
ResidueArray[10][4]:=Residues;

In [ ]:
for i in Keys(ResidueArray) do
    printf "singularity t=%o\n", i;
    for j in Keys(ResidueArray[i]) do
        r:=ResidueArray[i][j];
        printf "residues on component %o:\n", j;
        printf "x-p:\n";
        r[1];
        printf "\nx-q:\n";
        r[2];
        printf "\n";
    end for;
end for;